### Polynomial

1. Definition: $A_n(x) = \sum_{i=0}^{n-1} a_i x_i$.
2. Operations:  
\begin{align}
evaluate(A, x_0) = A(x_0), O(n) \\
add(A, B) = A(x) + B(x), O(n) \\
multiply(A, B) = A(x) \cdot B(x), O(nlogn)
\end{align}
3. multiplication of $A$ and $B$ is equivalent to a convolution of the vectors $A$ and $reverse(B)$. For example, $A = [1,2,3], B = [0,1,2]$:
```
---------------
|     [1 2 3] |
| [2 1 0]     |
---------------
|   [1 2 3]   |
| [2 1 0]     |
---------------
| [1 2 3]     |
| [2 1 0]     |
---------------
| [1 2 3]     |
|   [2 1 0]   |
---------------
| [1 2 3]     |
|     [2 1 0] |
---------------
```
4. FFT can convert coefficient vector form into sample form in $O(nlogn)$ time and IFFT in reverse.

### Fast Fourier Transform

Divide-and-Conquer:  
1. Divide the coefficient vector into odd and even entries: $A_{odd}(x)$ and $A_{even}(x)$.
2. Calculate the samples of $A_{odd}(x^2)$ and $A_{even}(x^2)$.
3. Merge samples: $A(x) = xA_{odd}(x^2) + A_{even}(x^2)$.

IDEA!: We should carefully choose the target sample set $X$, so that $|\{x^2 | x\in X\}| = \frac{1}{2}|X|$. How? Duplicate roots! $X = \{z | z^n = 1\}$.

### Inverse Fast Fourier Transform

$$
F_n = \frac{1}{\sqrt{n}}\begin{bmatrix}
    1 & 1 & 1 & \cdots & 1 \\
    1 & \omega & \omega^2 & \cdots & \omega^{n-1} \\
    1 & \omega^2 & \omega^4 & \cdots & \omega^{2(n-1)} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    1 & \omega^{n-1} & \omega^{2(n-1)} & \cdots & \omega^{(n-1)(n-1)}
\end{bmatrix}
$$
where $\omega=e^{\frac{2\pi i}{n}}$.

$A_n \cdot F_n = A^*_n$, where $A_n$ is the coefficient vector and $A^*_n$ is the sample vector of $y$ on $\{\omega^0, \omega^1, \cdots, \omega^{n-1}\}$.

Since $F_n \cdot \bar{F_n} = nI$, IFFT can be implemented by FFT.

### problem 2-1

1. For each substring $S[i :]$, we do pattern matching. The total time complexity is $O(mn)$.
2. For both $S$ and $P$, we apply the mapping: $f(a) = 1, f(b) = -1, f(*) = 0$. If the entry of $A_S \cdot reverse(A_P)$ equals to $count\_nonwild(P)$, then it's matched.  
For example, $S = ababbab, P = ab*$, we have $A_S = [1, -1, 1, -1, -1, 1, -1], A_P = [1, -1, 0]$. $A_M = A_S \cdot reverse(A_P) = [0, -1, 2, -2, 2, 0, -2]$, so $M = [0, 2]$.
3. First we extend $A_P$ and $A_S$ to $m+n$ degree (since the product of them is at most $m+n$ degree and we need $m+n$ samples), compute $A_S^* = FFT(A_S)$ and $A_P^* = FFT(A_P)$ in $O((m+n)log(m+n))$ time. Second, we compute their product in $O(m+n)$ time. Finally, apply IFFT to convert back in $O((m+n)log(m+n))$ time. The total time complexity is $O((m+n)log(m+n))$.
4. Orthogonal Basis: Similar to 2 but this time we have $4$ distinct symbols to match. We need $4$ numbers: $u_0, u_1, u_2, u_3$ that $u_i^2 = 1$ and $u_i \cdot u_j < 1, i \ne j$. Unfortunately, this is impossible at least in the real number domain.  
Another approach is Divide-and-Conquer: We apply two mappings separately to divide $4$ symbols into $2$ orthogonal channels.

| channel | A | C | G | T | * |
|---------|---|---|---|---|---|
| A       | 1 | 1 | -1 | -1 | 0 |
| B       | 1 | -1 | 1 | -1 | 0 |

Then we apply 2 to find $M_A$ and $M_B$ and the total result $M = M_A \cap M_B$. Brief Proof: If a position $i$ has value $1$ in channel A, then $S[i] = A | C$ and if so in channel B, then $S[i] = A | G$. Finally we get $S[i] = A$.

### problem 2-2

1. We create a new root based on the new key $k$. (Root should have at least $2$ children.)
```
    [k]    
   /   \   
(T1)   (T2)
```
2. We create a temporary root of $T_2$ based on the new key $k$ and try to combine $T_1$ and $T_2'$ (merge or borrow, depends on the children number of $T_1$'s root).
```
(T1) [k]    
        \   
        (T2)
```
3. Each node has a field $height$, denoting the height of its subtree. When $insert(T, key)$, if we create a new root, we calculate its $h$ (just add $1$ to its original height). When $delete(T, key)$, we do nothing on $h$.
4. Suppose $h_1 > h_2$, we append the new key $k$ to $T_2$, then we traverse $T_1$ and its right-most subtree until we find $h'_1 = h_2 + 1$. Insert $k$ and adjust this node up to root. The total time complexity is $O(|h_1-h_2| + 1)$.
```
... \          
... [... k]    
...        \   
...        (T2)
```